<a href="https://colab.research.google.com/github/alinemsnjc/5-Vb8nXmJ5Q7kZ/blob/main/C%C3%B3pia_de_Imers%C3%A3o_Agentes_de_IA_Alura_%2B_Google_Gemini_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 01

In [ ]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00


Importação da API Key

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Conexão com o Gemini

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [ ]:
resp_test = llm.invoke("Quem é você? Seja criativo.")
print(resp_test.content)

In [ ]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)

In [ ]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()

In [ ]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [ ]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

# Aula 02

In [ ]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

In [ ]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path("/content/").glob("*.pdf"):
    try:
        loader = PyMuPDFLoader(str(n))
        docs.extend(loader.load())
        print(f"Carregado com sucesso arquivo {n.name}")
    except Exception as e:
        print(f"Erro ao carregar arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = splitter.split_documents(docs)

Este código utiliza o RecursiveCharacterTextSplitter da biblioteca langchain_text_splitters para dividir os documentos carregados em pedaços menores (chunks).

Aqui está o que cada parte faz:

from langchain_text_splitters import RecursiveCharacterTextSplitter: Importa a classe RecursiveCharacterTextSplitter.
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30): Cria uma instância do splitter.
chunk_size=300: Define o tamanho máximo de cada pedaço em 300 caracteres.
chunk_overlap=30: Define uma sobreposição de 30 caracteres entre os pedaços. Isso ajuda a manter o contexto entre eles.
chunks = splitter.split_documents(docs): Divide a lista de documentos (docs) em pedaços menores (chunks) usando o splitter configurado.
Essa divisão é útil para processar documentos longos, pois permite que você trabalhe com partes menores do texto, o que é importante para muitas tarefas de processamento de linguagem natural, como a criação de embeddings para busca de similaridade.


Este trecho de código carrega documentos PDF do diretório /content/ usando o PyMuPDFLoader da biblioteca langchain_community.document_loaders.

Aqui está uma explicação detalhada:

from pathlib import Path: Importa a classe Path para trabalhar com caminhos de arquivo.
from langchain_community.document_loaders import PyMuPDFLoader: Importa o PyMuPDFLoader para carregar documentos PDF.
docs = []: Inicializa uma lista vazia para armazenar os documentos carregados.
for n in Path("/content/").glob("*.pdf"):: Este loop itera por todos os arquivos que terminam com .pdf no diretório /content/.
loader = PyMuPDFLoader(str(n)): Cria uma instância do PyMuPDFLoader para cada arquivo PDF encontrado.
docs.extend(loader.load()): Carrega o conteúdo do arquivo PDF usando o loader e o adiciona à lista docs.
print(f"Carregado com sucesso arquivo {n.name}"): Imprime uma mensagem de sucesso para cada arquivo carregado.
except Exception as e:: Este bloco lida com quaisquer erros que possam ocorrer durante o processo de carregamento.
print(f"Erro ao carregar arquivo {n.name}: {e}"): Imprime uma mensagem de erro se um arquivo não for carregado.
print(f"Total de documentos carregados: {len(docs)}"): Finalmente, imprime o número total de documentos carregados.

In [ ]:
for chunk in chunks:
    print(chunk)
    print("------------------------------------")

Este código simplesmente itera sobre a lista de "chunks" (os pedaços de texto gerados pelo RecursiveCharacterTextSplitter) e imprime o conteúdo de cada um, seguido por uma linha de separação (------------------------------------).

É útil para visualizar como os documentos foram divididos em pedaços menores e verificar se a divisão ocorreu conforme o esperado.

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=GOOGLE_API_KEY
)

Este código inicializa o modelo de embeddings do Google Gemini para criar representações vetoriais dos seus dados.

Aqui está o que cada parte faz:

from langchain_google_genai import GoogleGenerativeAIEmbeddings: Importa a classe GoogleGenerativeAIEmbeddings.
embeddings = GoogleGenerativeAIEmbeddings(...): Cria uma instância da classe GoogleGenerativeAIEmbeddings.
model="models/gemini-embedding-001": Especifica o modelo de embedding a ser utilizado, neste caso, o "gemini-embedding-001".
google_api_key=GOOGLE_API_KEY: Passa a sua chave de API do Google para autenticação. É importante que a variável GOOGLE_API_KEY contenha a sua chave de API carregada de forma segura (como feito anteriormente no notebook usando userdata.get).
O objeto embeddings criado pode ser usado para converter texto em vetores numéricos, que são essenciais para tarefas como busca de similaridade em bancos de dados vetoriais.



In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold":0.3, "k": 4})


Este código demonstra como criar um armazenamento vetorial (vectorstore) usando FAISS e configurar um retriever para buscar documentos relevantes com base em embeddings.

Aqui está o que cada parte faz:

from langchain_community.vectorstores import FAISS: Importa a classe FAISS da biblioteca langchain_community.vectorstores. FAISS é uma biblioteca para busca de similaridade eficiente e agrupamento de vetores densos.
vectorstore = FAISS.from_documents(chunks, embeddings): Cria uma instância do FAISS vectorstore a partir dos chunks (os pedaços de texto processados) e dos embeddings (as representações vetoriais desses chunks). Isso constrói o índice de busca no FAISS.
retriever = vectorstore.as_retriever(...): Converte o vectorstore FAISS em um objeto retriever. Um retriever é responsável por buscar documentos relevantes dado uma consulta.
search_type="similarity_score_threshold": Define o tipo de busca como busca por similaridade com um limite de pontuação. Isso significa que apenas documentos com uma pontuação de similaridade acima de um certo limite serão retornados.
search_kwargs={"score_threshold":0.3, "k": 4}: Define argumentos adicionais para a busca.
"score_threshold":0.3: Define o limite mínimo de pontuação de similaridade para que um documento seja considerado relevante. Documentos com pontuação abaixo de 0.3 serão ignorados.
"k": 4: Define o número máximo de documentos mais relevantes a serem retornados após a aplicação do limite de pontuação.
Em resumo, este código configura um sistema onde você pode dar uma consulta (pergunta), o retriever a converterá em um embedding, buscará no índice FAISS por chunks de documentos com embeddings semelhantes acima de um certo nível de similaridade, e retornará os chunks mais relevantes.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

Este código define um template de prompt para um modelo de linguagem e cria uma chain (cadeia) para combinar documentos com esse prompt.

Aqui está o que cada parte faz:

from langchain_core.prompts import ChatPromptTemplate: Importa a classe ChatPromptTemplate para criar prompts formatados para modelos de chat.
from langchain.chains.combine_documents import create_stuff_documents_chain: Importa a função create_stuff_documents_chain para criar uma chain que combina documentos com um prompt.
prompt_rag = ChatPromptTemplate.from_messages([...]): Cria uma instância de ChatPromptTemplate.
("system", ...): Define uma mensagem do sistema que instrui o modelo sobre seu papel e comportamento (ser um Assistente de Políticas Internas e responder apenas com base no contexto fornecido).
("human", "Pergunta: {input}\n\nContexto:\n{context}"): Define o formato da mensagem do usuário, incluindo um placeholder {input} para a pergunta do usuário e um placeholder {context} para o contexto relevante extraído dos documentos.
document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag): Cria a chain que combina os documentos. Ela recebe o modelo de linguagem (llm_triagem) e o template de prompt (prompt_rag). Quando esta chain é invocada, ela insere o contexto dos documentos relevantes no prompt e envia para o modelo de linguagem para gerar a resposta.
Em resumo, este código prepara o sistema para usar o contexto de documentos recuperados (pelo retriever) para responder a perguntas do usuário de forma informada.

In [ ]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [ ]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
    docs_relacionados = retriever.invoke(pergunta)

    if not docs_relacionados:
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    answer = document_chain.invoke({"input": pergunta,
                                    "context": docs_relacionados})

    txt = (answer or "").strip()

    if txt.rstrip(".!?") == "Não sei":
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    return {"answer": txt,
            "citacoes": formatar_citacoes(docs_relacionados, pergunta),
            "contexto_encontrado": True}

In [ ]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [ ]:
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"PERGUNTA: {msg_teste}")
    print(f"RESPOSTA: {resposta['answer']}")
    if resposta['contexto_encontrado']:
        print("CITAÇÕES:")
        for c in resposta['citacoes']:
            print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
            print(f"   Trecho: {c['trecho']}")
        print("------------------------------------")

# Aula 03

In [ ]:
!pip install -q --upgrade langgraph

Instalando o Langgraph

In [ ]:
from typing import TypedDict, Optional

class AgentState(TypedDict, total = False):
    pergunta: str
    triagem: dict
    resposta: Optional[str]
    citacoes: List[dict]
    rag_sucesso: bool
    acao_final: str

Este código define uma classe Python AgentState que é usada para representar o estado de um agente dentro de um fluxo de trabalho. Ele utiliza TypedDict do módulo typing para fornecer dicas de tipo, o que ajuda a entender a estrutura esperada do estado.

Aqui está um detalhe de cada parte:

from typing import TypedDict, Optional:
Esta linha importa TypedDict e Optional do módulo typing. TypedDict é usado para criar dicionários com chaves e tipos de valor específicos, e Optional indica que um valor pode ser de um tipo especificado ou None.
class AgentState(TypedDict, total=False)::
Isso define a classe AgentState, herdando de TypedDict. O argumento total=False significa que nem todas as chaves no dicionário são obrigatórias.
pergunta: str: Isso indica que o estado pode ter uma chave chamada pergunta que é esperada ser uma string.
triagem: dict: Isso indica uma chave chamada triagem que é esperada ser um dicionário.
resposta: Optional[str]: Isso indica uma chave chamada resposta que pode ser uma string ou None.
citacoes: List[dict]: Isso indica uma chave chamada citacoes que é esperada ser uma lista de dicionários.
rag_sucesso: bool:
 Isso indica uma chave chamada rag_sucesso que é esperada ser um valor booleano.
acao_final: str: Isso indica uma chave chamada acao_final que é esperada ser uma string.
Em resumo, a classe AgentState funciona como um modelo para um dicionário que armazena várias informações relacionadas ao processo do agente, como a pergunta do usuário (pergunta), o resultado de uma etapa de triagem (triagem), a resposta gerada (resposta), citações relevantes (citacoes), se uma etapa RAG (Retrieval Augmented Generation) foi bem-sucedida (rag_sucesso) e a ação final tomada (acao_final).

In [ ]:
def node_triagem(state: AgentState) -> AgentState:
    print("Executando nó de triagem...")
    return {"triagem": triagem(state["pergunta"])}

Esta função parece ser um nó em um fluxo de trabalho ou grafo de estado, como o que você pode construir com a biblioteca langgraph que foi mencionada anteriormente.

Aqui está o que a função faz:

def node_triagem(state: AgentState) -> AgentState:: Define a função chamada node_triagem que aceita um argumento state do tipo AgentState (a classe que você definou anteriormente para representar o estado do agente) e retorna um valor do mesmo tipo AgentState.
print("Executando nó de triagem..."): Imprime uma mensagem no console indicando que este nó está sendo executado. Isso é útil para depuração e para acompanhar o fluxo do programa.
return {"triagem": triagem(state["pergunta"])}: Esta é a parte principal da função.
Ele chama a função triagem (que você definiu em uma célula anterior para realizar a triagem da pergunta do usuário).
Passa a state["pergunta"] como argumento para a função triagem. Isso significa que a função de triagem usará a pergunta armazenada no estado atual do agente.
Retorna um dicionário contendo a chave "triagem" com o resultado da execução da função triagem. Este dicionário será usado para atualizar o estado do agente no fluxo de trabalho.
Em resumo, esta função node_triagem pega o estado atual do agente, extrai a pergunta, passa essa pergunta para a função de triagem, e retorna o resultado da triagem para ser adicionado ao estado do agente. É um passo fundamental na sua pipeline para determinar a natureza da solicitação do usuário.



Como mencionei anteriormente, os próximos passos no seu notebook envolveriam:

Definir as funções restantes para os nós do seu grafo Langgraph:
 Você já tem a função node_triagem. Provavelmente precisará de funções para lidar com o RAG (Retrieval Augmented Generation), talvez uma para lidar com casos que precisam de mais informações (PEDIR_INFO), e outra para formatar a resposta final.
Construir e compilar o grafo Langgraph:
 Conectar as funções definidas em um fluxo lógico usando a estrutura do langgraph. Isso define como o estado do agente muda entre os diferentes nós com base nas decisões tomadas (por exemplo, o resultado da triagem).
Executar e testar o grafo completo: Testar o fluxo de ponta a ponta com diferentes tipos de perguntas para garantir que o agente está se comportando conforme o esperado e tomando as ações corretas em cada cenário.

In [ ]:
def node_auto_resolver(state: AgentState) -> AgentState:
    print("Executando nó de auto_resolver...")
    resposta_rag = perguntar_politica_RAG(state["pergunta"])

    update: AgentState = {
        "resposta": resposta_rag["answer"],
        "citacoes": resposta_rag.get("citacoes", []),
        "rag_sucesso": resposta_rag["contexto_encontrado"],
    }

    if resposta_rag["contexto_encontrado"]:
        update["acao_final"] = "AUTO_RESOLVER"

    return update

In [ ]:
def node_pedir_info(state: AgentState) -> AgentState:
    print("Executando nó de pedir_info...")
    faltantes = state["triagem"].get("campos_faltantes", [])
    if faltantes:
        detalhe = ",".join(faltantes)
    else:
        detalhe = "Tema e contexto específico"

    return {
        "resposta": f"Para avançar, preciso que detalhe: {detalhe}",
        "citacoes": [],
        "acao_final": "PEDIR_INFO"
    }

In [ ]:
def node_abrir_chamado(state: AgentState) -> AgentState:
    print("Executando nó de abrir_chamado...")
    triagem = state["triagem"]

    return {
        "resposta": f"Abrindo chamado com urgência {triagem['urgencia']}. Descrição: {state['pergunta'][:140]}",
        "citacoes": [],
        "acao_final": "ABRIR_CHAMADO"
    }

In [ ]:
KEYWORDS_ABRIR_TICKET = ["aprovação", "exceção", "liberação", "abrir ticket", "abrir chamado", "acesso especial"]

def decidir_pos_triagem(state: AgentState) -> str:
    print("Decidindo após a triagem...")
    decisao = state["triagem"]["decisao"]

    if decisao == "AUTO_RESOLVER": return "auto"
    if decisao == "PEDIR_INFO": return "info"
    if decisao == "ABRIR_CHAMADO": return "chamado"

In [ ]:
def decidir_pos_auto_resolver(state: AgentState) -> str:
    print("Decidindo após o auto_resolver...")

    if state.get("rag_sucesso"):
        print("Rag com sucesso, finalizando o fluxo.")
        return "ok"

    state_da_pergunta = (state["pergunta"] or "").lower()

    if any(k in state_da_pergunta for k in KEYWORDS_ABRIR_TICKET):
        print("Rag falhou, mas foram encontradas keywords de abertura de ticket. Abrindo...")
        return "chamado"

    print("Rag falhou, sem keywords, vou pedir mais informações...")
    return "info"

In [ ]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(AgentState)

workflow.add_node("triagem", node_triagem)
workflow.add_node("auto_resolver", node_auto_resolver)
workflow.add_node("pedir_info", node_pedir_info)
workflow.add_node("abrir_chamado", node_abrir_chamado)

workflow.add_edge(START, "triagem")
workflow.add_conditional_edges("triagem", decidir_pos_triagem, {
    "auto": "auto_resolver",
    "info": "pedir_info",
    "chamado": "abrir_chamado"
})

workflow.add_conditional_edges("auto_resolver", decidir_pos_auto_resolver, {
    "info": "pedir_info",
    "chamado": "abrir_chamado",
    "ok": END
})

workflow.add_edge("pedir_info", END)
workflow.add_edge("abrir_chamado", END)

grafo = workflow.compile()

Ok, este código constrói e compila o grafo do Langgraph que define o fluxo do seu agente.

Vamos detalhar cada parte:

from langgraph.graph import StateGraph, START, END: Importa as classes e constantes necessárias da biblioteca langgraph.graph.
StateGraph: A classe principal para definir um grafo de estado.
START: Uma constante que representa o nó inicial do grafo.
END: Uma constante que representa o nó final do grafo.
workflow = StateGraph(AgentState): Cria uma instância de StateGraph. Ele recebe AgentState como argumento, indicando que este grafo operará sobre um estado que segue a estrutura definida pela classe AgentState que você criou anteriormente.
workflow.add_node("triagem", node_triagem): Adiciona um nó ao grafo.
O primeiro argumento "triagem" é o nome único deste nó dentro do grafo.
O segundo argumento node_triagem é a função Python que será executada quando o fluxo chegar a este nó. Esta é a função que você definiu anteriormente para realizar a triagem.
workflow.add_node("auto_resolver", node_auto_resolver): Adiciona o nó chamado "auto_resolver" associado à função node_auto_resolver.
workflow.add_node("pedir_info", node_pedir_info): Adiciona o nó chamado "pedir_info" associado à função node_pedir_info.
workflow.add_node("abrir_chamado", node_abrir_chamado): Adiciona o nó chamado "abrir_chamado" associado à função node_abrir_chamado.
workflow.add_edge(START, "triagem"): Define uma aresta (transição) no grafo.
Isso especifica que o fluxo começa no ponto START e transita diretamente para o nó "triagem".
workflow.add_conditional_edges("triagem", decidir_pos_triagem, { ... }): Adiciona arestas condicionais a partir do nó "triagem".
O primeiro argumento "triagem" é o nó de onde as transições partem.
O segundo argumento decidir_pos_triagem é a função "decisora". Esta função (que você definiu anteriormente) examinará o estado atual e retornará uma string que determina para qual nó o fluxo deve ir em seguida.
O terceiro argumento é um dicionário que mapeia os possíveis retornos da função decidir_pos_triagem para os nomes dos nós de destino.
Se decidir_pos_triagem retornar "auto", o fluxo vai para "auto_resolver".
Se retornar "info", vai para "pedir_info".
Se retornar "chamado", vai para "abrir_chamado".
workflow.add_conditional_edges("auto_resolver", decidir_pos_auto_resolver, { ... }): Adiciona arestas condicionais a partir do nó "auto_resolver".
Usa a função decisora decidir_pos_auto_resolver para determinar o próximo nó com base no resultado do processo de auto-resolução (RAG).
As transições possíveis são para "pedir_info", "abrir_chamado", ou END (fim do fluxo), dependendo do retorno da função decisora.
workflow.add_edge("pedir_info", END): Adiciona uma aresta que leva do nó "pedir_info" diretamente para o fim do fluxo (END).
workflow.add_edge("abrir_chamado", END): Adiciona uma aresta que leva do nó "abrir_chamado" diretamente para o fim do fluxo (END).
grafo = workflow.compile(): Compila o grafo que foi definido. Este passo finaliza a estrutura do grafo e o prepara para ser executado. O objeto grafo resultante é o que você usará para invocar o fluxo do agente com uma pergunta inicial.
Em resumo, este código monta a arquitetura do seu agente, definindo os passos (nós) e as regras para mover entre eles (arestas e arestas condicionais), criando um fluxo lógico para processar as perguntas do usuário.

In [ ]:
from IPython.display import display, Image

graph_bytes = grafo.get_graph().draw_mermaid_png()
display(Image(graph_bytes))

 Este código utiliza a biblioteca langgraph para gerar uma representação visual do grafo que você acabou de construir e a exibe no notebook.

Aqui está o que cada parte faz:

from IPython.display import display, Image: Importa as funções display e Image do módulo IPython.display. Estas funções são usadas no ambiente Colab (e Jupyter) para exibir objetos de forma rica, como imagens.
graph_bytes = grafo.get_graph().draw_mermaid_png(): Esta é a parte principal que gera a imagem do grafo.
grafo.get_graph(): A partir do objeto grafo compilado, este método obtém a representação interna do grafo.
.draw_mermaid_png(): Este método, disponível na representação do grafo, gera uma imagem do grafo no formato PNG, utilizando a sintaxe do Mermaid (uma linguagem de diagrama e visualização). A imagem é retornada como bytes.
display(Image(graph_bytes)): Exibe a imagem gerada no notebook.
Image(graph_bytes): Cria um objeto Image a partir dos bytes da imagem PNG gerada.
display(...): A função display renderiza este objeto Image no output da célula.
Em resumo, este código é uma ferramenta visual útil para verificar a estrutura do grafo que você definiu, garantindo que as arestas e nós estejam conectados da forma esperada antes de executá-lo. A visualização em formato Mermaid PNG torna o fluxo do agente mais fácil de entender e depurar.

In [ ]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "É possível reembolsar certificações do Google Cloud?",
          "Posso obter o Google Gemini de graça?",
          "Qual é a palavra-chave da aula de hoje?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [ ]:
for msg_test in testes:
    resposta_final = grafo.invoke({"pergunta": msg_test})

    triag = resposta_final.get("triagem", {})
    print(f"PERGUNTA: {msg_test}")
    print(f"DECISÃO: {triag.get('decisao')} | URGÊNCIA: {triag.get('urgencia')} | AÇÃO FINAL: {resposta_final.get('acao_final')}")
    print(f"RESPOSTA: {resposta_final.get('resposta')}")
    if resposta_final.get("citacoes"):
        print("CITAÇÕES:")
        for citacao in resposta_final.get("citacoes"):
            print(f" - Documento: {citacao['documento']}, Página: {citacao['pagina']}")
            print(f"   Trecho: {citacao['trecho']}")

    print("------------------------------------")

O código selecionado executa o grafo do Langgraph que você construiu com uma lista de perguntas de teste e imprime os resultados.

Vamos detalhar o que cada parte faz:

for msg_test in testes:: Este loop itera sobre a lista testes que contém várias strings, onde cada string é uma pergunta de teste.
resposta_final = grafo.invoke({"pergunta": msg_test}): Esta é a linha crucial onde o grafo é executado.
grafo.invoke(...): Invoca o grafo compilado.
{"pergunta": msg_test}: Passa o estado inicial para o grafo. Neste caso, o estado inicial é um dicionário com a chave "pergunta" cujo valor é a pergunta de teste atual (msg_test). O grafo começa a processar a partir do ponto START com este estado inicial.
resposta_final: Armazena o estado final do grafo após a execução. Este estado final conterá todas as atualizações feitas pelos nós do grafo (como o resultado da triagem, a resposta gerada, etc.).
triag = resposta_final.get("triagem", {}): Tenta obter o resultado da triagem do estado final (resposta_final). Se a chave "triagem" não existir no estado final (o que não deve acontecer neste grafo, mas é uma boa prática usar .get()), ele retorna um dicionário vazio {} como padrão.
print(f"PERGUNTA: {msg_test}"): Imprime a pergunta de teste que foi processada.
print(f"DECISÃO: {triag.get('decisao')} | URGÊNCIA: {triag.get('urgencia')} | AÇÃO FINAL: {resposta_final.get('acao_final')}"): Imprime informações importantes sobre o resultado da execução do grafo para aquela pergunta:
triag.get('decisao'): A decisão de triagem (AUTO_RESOLVER, PEDIR_INFO, ABRIR_CHAMADO).
triag.get('urgencia'): A urgência determinada pela triagem (BAIXA, MEDIA, ALTA).
resposta_final.get('acao_final'): A ação final tomada pelo grafo (AUTO_RESOLVER, PEDIR_INFO, ABRIR_CHAMADO), que é definida pelos nós finais do fluxo.
print(f"RESPOSTA: {resposta_final.get('resposta')}"): Imprime a resposta gerada pelo grafo, que pode ser a resposta do RAG, um pedido de mais informações, ou a confirmação de abertura de chamado.
if resposta_final.get("citacoes"): ...: Verifica se há citações no estado final (resposta_final). Se houver (resposta_final.get("citacoes") retornar uma lista não vazia), ele imprime as citações.
print("CITAÇÕES:"): Imprime o cabeçalho para a seção de citações.
for citacao in resposta_final.get("citacoes"): ...: Itera sobre a lista de citações.
print(f" - Documento: {citacao['documento']}, Página: {citacao['pagina']}"): Imprime o nome do documento e o número da página da citação.
print(f" Trecho: {citacao['trecho']}"): Imprime o trecho relevante do documento citado.
print("------------------------------------"): Imprime uma linha separadora para distinguir os resultados de cada pergunta de teste.
Em resumo, este código pega uma lista de perguntas, executa cada uma delas através do grafo do Langgraph, e depois imprime um resumo detalhado do resultado, incluindo a decisão de triagem, a ação final, a resposta gerada e, se aplicável, as citações relevantes dos documentos usados pelo RAG. Isso permite que você veja como o seu agente se comporta com diferentes tipos de entradas.

